In [2]:
import math
import numpy as np
import pandas as pd
import scipy
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt

import scipy.stats
from scipy import stats, optimize
from scipy.stats import genextreme

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler

sn.set(rc={'figure.figsize':(21, 9)})
%matplotlib notebook

In [3]:
#data = pd.read_csv('data-1.csv')
data = pd.read_csv('combine_partial_lag_0.csv')
#data = pd.read_csv('combine_partial_NoLag-1.csv')

In [4]:
data

,East_12,lag_east_12,13,14,15,16,17,18,19,20,...,49.2,50.2,51.2,52.2,53.2,54.2,55.2,56.2,57.2,58.2
0,-0.017703,-0.016553,0.021267,0.042634,0.082478,0.130230,0.139420,0.141490,0.12413,0.14460,...,0.002730,0.001201,-0.001766,0.005172,0.004519,0.006129,0.004212,0.004655,0.001408,0.003451
1,-0.001151,-0.028014,0.027178,0.053463,0.090675,0.120980,0.137650,0.148710,0.13235,0.13501,...,0.002280,-0.000927,-0.002955,0.004738,0.002229,0.001678,-0.002080,0.002313,-0.000923,0.004295
2,0.026863,0.010981,0.039995,0.089400,0.115810,0.121480,0.137530,0.156670,0.14227,0.13988,...,0.001380,0.000830,-0.001670,0.001650,-0.000317,-0.003118,-0.006825,-0.001222,-0.003579,-0.002215
3,0.015882,0.039388,0.027085,0.082073,0.115360,0.133080,0.152590,0.178490,0.17368,0.18923,...,-0.000976,0.001851,-0.000283,-0.001392,-0.001417,-0.005652,-0.007379,-0.001470,-0.001904,-0.003790
4,-0.023506,0.011090,-0.003510,0.034058,0.056934,0.098202,0.130510,0.160280,0.18006,0.21011,...,-0.000047,0.000326,-0.001703,-0.001424,-0.002720,-0.006904,-0.006104,-0.001440,-0.001904,-0.004034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,-0.037374,0.021999,-0.046919,-0.026823,0.021059,0.050554,0.109600,0.133590,0.18231,0.23533,...,0.002944,0.003265,0.005050,0.002603,0.001479,0.004387,0.001433,0.001364,0.001453,0.003321
1596,-0.059373,0.014265,-0.065855,-0.060359,-0.003867,0.030398,0.079186,0.097516,0.16213,0.21842,...,0.001534,0.002233,0.004449,-0.001691,0.000040,0.001875,-0.000201,0.000972,0.001081,0.002468
1597,-0.073638,-0.046953,-0.066995,-0.058604,-0.013420,0.017367,0.059109,0.078131,0.13627,0.18862,...,0.002072,0.000454,0.002860,0.000392,-0.000318,0.001540,0.000687,-0.002021,-0.003631,-0.002200
1598,-0.026685,-0.020820,-0.006613,0.019691,0.058844,0.072697,0.120290,0.140470,0.17203,0.21967,...,-0.000649,0.000430,0.001610,-0.000674,-0.002238,0.000660,-0.002738,-0.007083,-0.006382,-0.004409


In [5]:
data_values = data.values
features = data_values[:,1:]
values = data_values[:,0]

In [6]:
features

array([[-1.65525e-02,  2.12670e-02,  4.26340e-02, ...,  4.65540e-03,
         1.40840e-03,  3.45120e-03],
       [-2.80135e-02,  2.71780e-02,  5.34630e-02, ...,  2.31340e-03,
        -9.23040e-04,  4.29470e-03],
       [ 1.09810e-02,  3.99950e-02,  8.94000e-02, ..., -1.22200e-03,
        -3.57900e-03, -2.21460e-03],
       ...,
       [-4.69530e-02, -6.69950e-02, -5.86040e-02, ..., -2.02140e-03,
        -3.63100e-03, -2.20050e-03],
       [-2.08203e-02, -6.61340e-03,  1.96910e-02, ..., -7.08350e-03,
        -6.38210e-03, -4.40870e-03],
       [ 1.75123e-02,  2.13290e-02,  4.23470e-02, ..., -4.51130e-03,
        -3.11520e-03,  9.35000e-05]])

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100,
       subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
       min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None,
       init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, 
       warm_start=False, presort='auto', validation_fraction=0.1, tol=0.0001)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

clf2 = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [9]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score
# arr_r2 = []

# for i in range(10):
    
#     X_train, X_test, Y_train, Y_test = train_test_split(features, values,test_size=0.3)
#     clf.fit(X_train, Y_train, sample_weight=None)
#     predict_Y = clf.predict(X_test)
#     arr_r2.append(r2_score(Y_test, predict_Y))
    

# arr_r2
# np.mean(arr_r2)

In [10]:
clf.fit(features, values, sample_weight=None)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
# clf2.fit(features, values, sample_weight=None)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [15]:
#test_data = pd.read_csv('data-2.csv')
test_data = pd.read_csv('combine_partial_lag_1.csv')
#test_data = pd.read_csv('combine_partial_NoLag-2.csv')
#test_data = test_data.head(500) 

In [16]:
test_data_values = test_data.values
test_features = test_data_values[:,1:]
test_values = test_data_values[:,0]

In [17]:
pred1 = clf.predict(test_features)

In [18]:
#pred2 = clf2.predict(test_features)

In [19]:
from sklearn.metrics import r2_score
r2_score(test_values, pred1)

0.9850742665988801

In [20]:
# from sklearn.metrics import r2_score
# r2_score(test_values, pred2)

0.9864864183495504